In [196]:
import numpy as np
import re
from bs4 import BeautifulSoup
import pandas as pd
import requests
import scipy.interpolate as sp

In [199]:
content = "https://weather.com/weather/tenday/l/8c6b0e55d5cf8568f60d839eaf3fa128975a8daf414f334c76ea19e9e1e1d3b0"
response = requests.get(content)
soup = BeautifulSoup(response.content, 'html.parser')

In [203]:
temp_val = soup.findAll('div', class_ ='DailyForecast--DisclosureList--nosQS')
next13 = temp_val[0].text.split('|')[-26:]
temp = [float(re.findall(r'\d+', next13[i].split('.')[0])[0]) for i in range(0, len(next13))]
humid = [float(re.findall(r'\d+', next13[i].split('.')[3])[0]) for i in range(0, len(next13))]

hour, hours = np.arange(12, 27*12, 12), np.arange(12, 27*12, 1)
t = sp.interp1d(hour, temp, kind='linear', fill_value = 'extrapolate')
h = sp.interp1d(hour, humid, kind='linear', fill_value = 'extrapolate')
temperature, humidity = t(hours), h(hours)

forecast = pd.DataFrame()
forecast['hours'] = hours
forecast['temperature'] = temperature
forecast['humidity'] = humidity
forecast['wind'] = np.concatenate([float(re.findall(r'\d+', next13[i].split('.')[0])[2])*np.ones(12) for i in range(0, len(next13))])
forecast['rain'] = np.concatenate([float(re.findall(r'\d+', next13[i].split('.')[0])[1])*np.ones(12) for i in range(0, len(next13))])
